In [1]:
import pandas as pd 
import numpy as np 
from pathlib import Path
from sklearn.preprocessing import OneHotEncoder

In [2]:
%%time
data_path = Path('./PrecisionFDA/train')                                                                                                                 
conditions = pd.read_csv( data_path / Path('conditions.csv'))                                                                                                                    
medications = pd.read_csv(data_path / Path('medications.csv'))                                                                                                                   
patients = pd.read_csv(data_path / Path('patients.csv'))
observations= pd.read_csv(data_path / Path('observations.csv'))
procedures= pd.read_csv(data_path / Path('procedures.csv'))


cond_cols    = ['START', 'PATIENT', 'CODE']
med_cols     = ['START', 'PATIENT', 'CODE']
patient_cols = ['Id', 'BIRTHDATE', 'DEATHDATE', 'GENDER', 'RACE', 'ETHNICITY']
obs_cols     = ['DATE', 'PATIENT', 'CODE', 'UNITS', 'VALUE', 'TYPE']
proc_cols    = ['DATE', 'PATIENT', 'CODE']

cov_id = conditions[conditions['CODE'] == 840539006].PATIENT
patients      = patients[patient_cols]
conditions    = conditions[cond_cols]
medications   = medications[med_cols]
observations  = observations[obs_cols]
procedures    = procedures[proc_cols]

# Rename some duplicate columns
medications = medications.rename(columns={'PATIENT':'Id', 'START':'START_med', 'CODE':'CODE_med'})
conditions  = conditions.rename(columns={'PATIENT':'Id', 'START':'START_cond', 'CODE':'CODE_cond'})
observations  = observations.rename(columns={'PATIENT':'Id', 'DATE':'DATE_obs', 'CODE':'CODE_obs'})
procedures  = procedures.rename(columns={'PATIENT':'Id', 'DATE':'DATE_proc', 'CODE':'CODE_proc'})

medications.CODE_med = medications.CODE_med.astype(str)
conditions.CODE_cond = conditions.CODE_cond.astype(str)
observations.CODE_obs = observations.CODE_obs.astype(str)
procedures.CODE_proc = procedures.CODE_proc.astype(str)

medications['START_med' ]   = pd.to_datetime(medications['START_med'])
conditions ['START_cond']   = pd.to_datetime(conditions['START_cond'])
observations['DATE_obs']    = pd.to_datetime(observations['DATE_obs'])
procedures['DATE_proc']     = pd.to_datetime(procedures['DATE_proc'])


patients   ['BIRTHDATE' ] = pd.to_datetime(patients['BIRTHDATE'])
patients   ['DEATHDATE' ] = pd.to_datetime(patients['DEATHDATE'])



Wall time: 1min 26s


In [4]:
pat_cols = ['Id', 'BIRTHDATE']
patients_hos = patients[pat_cols]
print(patients_hos.shape)
medications = medications.merge(patients_hos, on='Id', how='outer')
conditions = conditions.merge(patients_hos, on='Id', how='outer')
observations = observations.merge(patients_hos, on='Id', how='outer')
procedures = procedures.merge(patients_hos, on='Id', how='outer')

print(medications.shape)
print(conditions.shape)
print(observations.shape)
print(procedures.shape)

(117959, 2)
(9681137, 4)
(1362934, 4)
(20656425, 7)
(1304838, 4)


In [5]:
print(len(medications['Id'].unique()))
print(len(conditions['Id'].unique()))
print(len(patients['Id'].unique()))

117959
117959
117959


In [6]:
encounters = pd.read_csv( data_path / Path('encounters.csv'))   
enc_cols = ['START', 'STOP', 'PATIENT', 'CODE', 'REASONCODE']
encounters = encounters[enc_cols]
encounters = encounters.rename(columns={'PATIENT': 'Id'})
encounters['START'] = pd.to_datetime(encounters['START'])
encounters['STOP'] = pd.to_datetime(encounters['STOP'])

In [7]:
#Days in hospital
#y_encounters = encounters[(encounters.CODE == 1505002) & (encounters.REASONCODE == 840539006)]

# Days ICU
y_encounters = encounters[(encounters.CODE == 305351004)]

y_encounters = y_encounters[y_encounters['Id'].isin(cov_id)]
y_encounters['durration'] = round((y_encounters['STOP'] - y_encounters['START']) / pd.to_timedelta(1, unit='D'), 2)
y_encounters = y_encounters.drop(columns=['START', 'STOP', 'REASONCODE', 'CODE'])


In [8]:
print(y_encounters)

                                           Id  durration
296      c58ae02e-235e-4db2-a9c3-6926e9c0c5fa       7.13
1920     52e1f151-e98b-4163-a250-5e7d5c6ff813       9.40
2918     534e1ef3-b3c7-42da-a022-fea54d6c828f       7.15
3041     5158d043-8c93-4987-8fcb-5f94e7e32fa1       5.81
3607     b5499d4a-6660-4b09-bfc0-e3d33e3dcb5e       7.94
...                                       ...        ...
6018069  9c92df1f-e325-43c2-9bc0-1380eeaf3958       6.69
6018396  6cda9ea0-5264-4465-9a9c-8315994a488f       5.94
6021798  9529dc13-c7f1-4f76-b903-4e310b7805c8       4.18
6022463  65d3785a-6150-4b86-8842-c40ca654e3c4       9.91
6022720  8722685f-ce37-4520-ab95-22b6929526c2       3.25

[4981 rows x 2 columns]


In [9]:
# Turn Dates into ages
medications['Age'] = round((medications['START_med'] - medications['BIRTHDATE']) / pd.to_timedelta(1, unit='D'), 2)
medications = medications.drop(columns=['START_med', 'BIRTHDATE'])
print(medications.head())

conditions['Age'] = round((conditions['START_cond'] - conditions['BIRTHDATE']) / pd.to_timedelta(1, unit='D'), 2)
conditions = conditions.drop(columns=['START_cond', 'BIRTHDATE'])
print(conditions.head())

observations['Age'] = round((observations['DATE_obs'] - observations['BIRTHDATE']) / pd.to_timedelta(1, unit='D'), 2)
observations = observations.drop(columns=['DATE_obs', 'BIRTHDATE', 'TYPE', 'UNITS'])
print(observations.head())

procedures['Age'] = round((procedures['DATE_proc'] - procedures['BIRTHDATE']) / pd.to_timedelta(1, unit='D'), 2)
procedures = procedures.drop(columns=['DATE_proc', 'BIRTHDATE'])
print(procedures.head())


                                     Id CODE_med     Age
0  93fa6213-73a1-4cce-b408-9986a4145801   312961  2177.0
1  93fa6213-73a1-4cce-b408-9986a4145801   197361  2177.0
2  93fa6213-73a1-4cce-b408-9986a4145801   312961  2177.0
3  93fa6213-73a1-4cce-b408-9986a4145801   197361  2177.0
4  93fa6213-73a1-4cce-b408-9986a4145801   312961  2548.0
                                     Id  CODE_cond     Age
0  93fa6213-73a1-4cce-b408-9986a4145801   53741008  2177.0
1  93fa6213-73a1-4cce-b408-9986a4145801  195662009  4124.0
2  93fa6213-73a1-4cce-b408-9986a4145801   22298006  4473.0
3  93fa6213-73a1-4cce-b408-9986a4145801  399211009  4473.0
4  53bbfa41-5cab-422e-a8a7-8fa3ae55ed71  370143000  6985.0
                                     Id CODE_obs  VALUE     Age
0  93fa6213-73a1-4cce-b408-9986a4145801   8310-5   37.9  4124.0
1  93fa6213-73a1-4cce-b408-9986a4145801   8302-2  141.0  4403.0
2  93fa6213-73a1-4cce-b408-9986a4145801  72514-3    1.0  4403.0
3  93fa6213-73a1-4cce-b408-9986a4145801  29463-7

In [10]:
df_meds = pd.get_dummies(medications, columns=['CODE_med'], prefix='Meds')
df_copyM = df_meds
print(len(df_meds['Id'].unique()))
df_meds = df_meds.groupby('Id').max()
print(df_meds.head())


df_conds = pd.get_dummies(conditions, columns=['CODE_cond'], prefix='Conds')
df_copyC = df_conds
df_conds = df_conds.groupby('Id').max()
print(df_conds.head())


df_proc = pd.get_dummies(procedures, columns=['CODE_proc'], prefix='Proc')
df_copyP = df_proc
df_proc = df_proc.groupby('Id').max()
print(df_proc.head())

df_ob = pd.get_dummies(observations, columns=['CODE_obs'], prefix='Obs')
df_copyOb = df_ob
df_ob = df_ob.groupby('Id').max()
print(df_ob.head())

117959
                                          Age  Meds_1000126  Meds_1014678  \
Id                                                                          
0000641f-540d-408b-ad16-93c2de94f446  17213.0             0             0   
0000ae6a-e3bd-4136-98e0-6d99a3ca08ad  17507.0             0             0   
00018c7f-b75e-4d55-a05a-ba3b3372f910  22715.0             0             0   
00019696-c3de-4e1f-8ca8-0df186aa988c   9740.0             0             0   
00028c0a-864f-4183-90d5-c703edf779d0  24461.0             0             0   

                                      Meds_1043400  Meds_1049221  \
Id                                                                 
0000641f-540d-408b-ad16-93c2de94f446             0             0   
0000ae6a-e3bd-4136-98e0-6d99a3ca08ad             0             0   
00018c7f-b75e-4d55-a05a-ba3b3372f910             0             0   
00019696-c3de-4e1f-8ca8-0df186aa988c             0             0   
00028c0a-864f-4183-90d5-c703edf779d0         

                                          Age  Proc_103746007  Proc_10383002  \
Id                                                                             
0000641f-540d-408b-ad16-93c2de94f446  17332.0               0              0   
0000ae6a-e3bd-4136-98e0-6d99a3ca08ad  17507.0               0              0   
00018c7f-b75e-4d55-a05a-ba3b3372f910      NaN               0              0   
00019696-c3de-4e1f-8ca8-0df186aa988c   9745.0               0              0   
00028c0a-864f-4183-90d5-c703edf779d0  24461.0               0              0   

                                      Proc_104091002  Proc_104173009  \
Id                                                                     
0000641f-540d-408b-ad16-93c2de94f446               0               0   
0000ae6a-e3bd-4136-98e0-6d99a3ca08ad               0               0   
00018c7f-b75e-4d55-a05a-ba3b3372f910               0               0   
00019696-c3de-4e1f-8ca8-0df186aa988c               0               0   
00028c0

In [11]:
df_meds = df_meds.drop(columns='Age')
df_conds = df_conds.drop(columns='Age')
df_proc = df_proc.drop(columns='Age')
df_ob = df_ob.drop(columns='Age')

In [13]:
print(medications['Id'].unique())
df = pd.pivot_table(medications,index="Id",columns="CODE_med", values='Age', aggfunc=np.max)
print(df.shape)
print(df.index)

df_C = conditions.pivot_table(index="Id",columns="CODE_cond", values='Age', aggfunc=np.max)
print(df_C.head())

df_P = procedures.pivot_table(index="Id",columns="CODE_proc", values='Age', aggfunc=np.max)
print(df_P.head())

df_obs_two = observations.pivot_table(index="Id",columns="CODE_obs", values='Age', aggfunc=np.max)



['93fa6213-73a1-4cce-b408-9986a4145801'
 '30fb496e-b8d3-4d36-8e50-86acb3d1a223'
 'afa2680f-7f73-46d9-b0cd-2cf3db49724b' ...
 '446aa4b9-c2bf-41fc-94c4-db1afaf8b423'
 '22a827ec-272f-4798-941a-a94c13b7c1fe'
 '3331c4dd-9378-499b-86e4-4d75d649b03a']
(107365, 182)
Index(['0000641f-540d-408b-ad16-93c2de94f446',
       '0000ae6a-e3bd-4136-98e0-6d99a3ca08ad',
       '00018c7f-b75e-4d55-a05a-ba3b3372f910',
       '00019696-c3de-4e1f-8ca8-0df186aa988c',
       '00028c0a-864f-4183-90d5-c703edf779d0',
       '000309a5-2940-4366-b320-b0cb90ac6dfe',
       '000371b6-0650-4946-9659-d504b7107ca6',
       '0003bf3b-f6f7-4ca9-b34a-db784779089a',
       '00069e22-85e6-4f52-a940-5e725b0ef71b',
       '0006bbf1-0bb7-480c-88c8-4e064bd20d98',
       ...
       'fffbdc12-79ad-48b7-bb32-2e10f5262373',
       'fffbe496-251d-4b66-9142-6baab76ce2d4',
       'fffc30e3-fe5f-460f-98cc-7110625e4a67',
       'fffcab67-0d65-47f6-a6bd-d21371a0d186',
       'fffcc1f0-7e09-4539-a57c-7bda7c9f0cf1',
       'fffe583e-9fe6-458

In [14]:
final_meds = df.merge(df_meds, on='Id')
print(final_meds.shape)

final_conds = df_C.merge(df_conds, on='Id')
print(final_conds.shape)

final_procs = df_P.merge(df_proc, on='Id')
print(final_procs.shape)

final_obs = df_obs_two.merge(df_ob, on='Id')
print(final_obs.shape)

final_events = final_meds.merge(final_conds, on='Id', how='outer').merge(final_procs, on='Id', how='outer')#.merge(final_obs, on='Id', how='outer')
print(final_events.shape)
print(final_events.head())

(107365, 364)
(117626, 380)
(111735, 350)
(117959, 431)
(117788, 1094)
                                      1000126  1014678  1043400  1049221  \
Id                                                                         
0000641f-540d-408b-ad16-93c2de94f446      NaN      NaN      NaN      NaN   
0000ae6a-e3bd-4136-98e0-6d99a3ca08ad      NaN      NaN      NaN      NaN   
00018c7f-b75e-4d55-a05a-ba3b3372f910      NaN      NaN      NaN      NaN   
00019696-c3de-4e1f-8ca8-0df186aa988c      NaN      NaN      NaN      NaN   
00028c0a-864f-4183-90d5-c703edf779d0      NaN      NaN      NaN      NaN   

                                      1049630  1049635  105078  105585  \
Id                                                                       
0000641f-540d-408b-ad16-93c2de94f446      NaN      NaN     NaN     NaN   
0000ae6a-e3bd-4136-98e0-6d99a3ca08ad   1846.0      NaN     NaN     NaN   
00018c7f-b75e-4d55-a05a-ba3b3372f910      NaN      NaN     NaN     NaN   
00019696-c3de-4e1f-8ca8-0d

In [17]:
print(patients.shape)
dead_people = patients[patients.DEATHDATE.notnull()].Id
dead_enc = encounters[encounters.Id.isin(dead_people)]
alive_enc = encounters[~encounters.Id.isin(dead_people)]

#Uncomment for getting training set to predict hospital stay
# hosp_dead = dead_enc[(dead_enc.REASONCODE == 840539006) & (dead_enc.CODE == 1505002)]
# hosp_dead['event'] = 0
# hosp_dead = hosp_dead.drop(columns = ['START', 'STOP', 'CODE', 'REASONCODE'])
# hosp_alive = alive_enc[(alive_enc.REASONCODE == 840539006) & (alive_enc.CODE == 1505002)]
# hosp_alive['event'] = 1
# hosp_alive = hosp_alive.drop(columns = ['START', 'STOP', 'CODE', 'REASONCODE'])
# enc_events = pd.concat([hosp_alive, hosp_dead])


# Uncomment for training set to predict ICU stay
ICU_dead = dead_enc[(dead_enc.CODE == 305351004)]
ICU_dead['event'] = 0
ICU_dead = ICU_dead.drop(columns = ['START', 'STOP', 'CODE', 'REASONCODE'])
hosp_alive = alive_enc[ (alive_enc.CODE == 305351004)]
hosp_alive['event'] = 1
hosp_alive = hosp_alive.drop(columns = ['START', 'STOP', 'CODE', 'REASONCODE'])
enc_events = pd.concat([hosp_alive, ICU_dead])


(117959, 6)


C:\Users\Konzelc\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\Konzelc\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [18]:
final_df = final_events.merge(y_encounters, on='Id', how='outer')

final_df = final_df.merge(enc_events, on='Id', how='outer')
final_df = final_df.fillna(0)
print(final_df.head())

                                     Id  1000126  1014678  1043400  1049221  \
0  0000641f-540d-408b-ad16-93c2de94f446      0.0      0.0      0.0      0.0   
1  0000ae6a-e3bd-4136-98e0-6d99a3ca08ad      0.0      0.0      0.0      0.0   
2  00018c7f-b75e-4d55-a05a-ba3b3372f910      0.0      0.0      0.0      0.0   
3  00019696-c3de-4e1f-8ca8-0df186aa988c      0.0      0.0      0.0      0.0   
4  00028c0a-864f-4183-90d5-c703edf779d0      0.0      0.0      0.0      0.0   

   1049630  1049635  105078  105585  106258  ...  Proc_82078001  \
0      0.0      0.0     0.0     0.0     0.0  ...            0.0   
1   1846.0      0.0     0.0     0.0     0.0  ...            0.0   
2      0.0      0.0     0.0     0.0     0.0  ...            0.0   
3      0.0      0.0     0.0     0.0     0.0  ...            0.0   
4      0.0      0.0     0.0     0.0     0.0  ...            0.0   

   Proc_85548006  Proc_88039007  Proc_88848003  Proc_90226004  Proc_90407005  \
0            0.0            0.0           

In [19]:
final_df.to_csv("pyCox_train_input_1000_ICU.csv")

,Id,1000126,1014678,1043400,1049221,1049630,1049635,105078,105585,106258,...,Proc_82078001,Proc_85548006,Proc_88039007,Proc_88848003,Proc_90226004,Proc_90407005,Proc_90470006,Proc_91602002,durration,event
144,00568ecc-34c8-463a-9de9-1a33456cb977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.02,1.0
201,00801ab7-f49f-4d2e-a9d9-61607086fb8e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.14,1.0
204,00813613-2791-4fc1-b79b-53fe18e5415b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.04,1.0
613,0177fec0-796d-45dc-88df-f0ad21e02446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10228.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.07,1.0
617,0179b373-c158-47cb-a639-6790f42bb967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.03,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106893,fed3ab94-8baf-4932-8389-2c695fcccddd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.12,1.0
107081,ff5335ac-d942-45d1-bbe3-222444d18f55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.03,1.0
107176,ff9784ad-3be8-478f-9bc3-498348786769,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.07,1.0
107190,ff9ed2b3-1cd9-4c78-9c0f-d310f183ef0a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.09,1.0
